# Barren Plateaus Numerical Study

In [42]:
from IPython.display import set_matplotlib_formats
import pandas as pd
import seaborn as sns
import glob, os
import numpy as np
from math import comb
import ast
import matplotlib.pyplot as plt
import matplotlib as mpl

#mpl.rcParams['font.family'] = 'serif'
#font_path = mpl.get_data_path() + '/fonts/ttf/cmr10.ttf'
#cmfont = mpl.font_manager.FontProperties(fname=font_path)
#mpl.rcParams['font.serif'] = cmfont.get_name()
#mpl.rcParams['font.size'] = 16
#mpl.rcParams['mathtext.fontset'] = 'cm'
#mpl.rcParams['axes.unicode_minus'] = False

%matplotlib inline
set_matplotlib_formats('pdf')

In [2]:
sns.set_theme(
    context=sns.plotting_context('talk'),
    style=sns.axes_style('white')
)
sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})

## Symmetry Preserving Circuits

In [17]:
#df = pd.read_csv('data/bp_discovery_output_spc_20210218_095941.dat')
df = pd.read_csv('data/bp_discovery_output_spc_20210218_131558.dat')
df['frac'] = df['m'] / df['n']
df = df[df['grad'] != 0.0]

In [18]:
agg = df.groupby(by=['ans', 'n', 'm']).agg({'grad': {'mean', 'var'}})
agg.columns = agg.columns.to_flat_index()
agg.reset_index(inplace=True)

agg['frac'] = agg['m'] / agg['n']

In [15]:
#df_input_spc_raw = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "data/spc/output_0.csv"))))
#df_input_spc_raw['ans'] = 'spc'
#df_input_spc_raw['frac'] = df_input_spc_raw['m'] / df_input_spc_raw['n']

In [16]:
sns.lineplot(
    x='frac',
    y=('grad', 'var'),
    hue='n',
    data=agg
)

<AxesSubplot:xlabel='frac', ylabel="('grad', 'var')">

<Figure size 432x288 with 1 Axes>

In [6]:
fig, axs = plt.subplots(ncols=1, nrows=2)

sns.lineplot(
    x='frac',
    y=('grad', 'var'),
    hue='n',
    data=agg,
    ax=axs[0]
)
axs[0].set_xlabel(r'$m / n$')
axs[0].set_ylabel(r'var$(\partial f)$')
axs[0].set_yscale('log')
axs[0].legend(loc='lower right')

axs[0].legend().remove()

_pars_dat = []
for n in range(4, 11, 2):
    for m in range(1, n):
        pars = 2*(comb(n, m) - 1)
        _pars_dat.append({
            'n': n,
            'm': m,
            'pars': pars,
            '1/pars': 1/pars,
            'frac': m/n
        })

df_pars = pd.DataFrame(_pars_dat)

sns.lineplot(
    x='frac',
    y='1/pars',
    hue='n',
    data=df_pars,
    ax=axs[1]
)

axs[0].set_xticks([])
axs[0].set_xlabel('')

axs[1].set_xlabel(r'$m / n$')
axs[1].set_ylabel(r'$1/(2\binom{n}{m}-2)$')
axs[1].set_yscale('log')
axs[1].legend(loc=(1.1, 0), title='Qubits')

<Figure size 432x288 with 2 Axes>

## V vs G vs Pauli Pools

In [29]:
df_mcp = pd.read_csv('data/bp_discovery_output_mcp_george_20210223_125358.dat')
df_mcp['pars'] = df_mcp['l']
df_mcp['ans'] = 'G'

df_vlad = pd.read_csv('data/bp_discovery_output_mcp_vlad_20210223_124712.dat')
df_vlad['pars'] = df_vlad['l']
df_vlad['ans'] = 'V'

df_full = pd.read_csv('data/bp_discovery_output_ocp_20210223_121516.dat')
df_full['pars'] = df_full['l']
df_full['ans'] = 'Full'

In [30]:
df = pd.concat([df_mcp, df_vlad, df_full])

agg = df.groupby(by=['ans', 'n', 'pars']).agg({'grad': {'mean', 'var'}})
agg.columns = agg.columns.to_flat_index()
agg.reset_index(inplace=True)

In [31]:
ax = sns.lineplot(
    data=agg,
    x='pars',
    y=('grad', 'var'),
    hue='n',
    style='ans'
)

legend = ax.get_legend()

leg_dict = {
    'n': 'Qubits',
    'ans': 'Ansatz',
    'hwe': 'HWE',
    'mcp': 'MCP',
    'ocp': 'OCP'
}

for text in legend.get_texts():
    s = text.get_text()
    if s in leg_dict:
        text.set_text(leg_dict[s])

ax.set_yscale('log')
ax.set_xlabel('Number of Parameters')
ax.set_ylabel(r'var$(\partial f)$')

Text(0, 0.5, 'var$(\\partial f)$')

<Figure size 432x288 with 1 Axes>

## MCP vs HWE

In [9]:
df_mcp = pd.read_csv('data/bp_discovery_output_mcp_20210216_150920.dat')
df_mcp['pars'] = df_mcp['l']

In [4]:
df_hwe = pd.read_csv('data/bp_discovery_output_hwe_20210216_103125.dat')
df_hwe = df_hwe[df_hwe['ans'] == 'hwe']
df_hwe['pars'] = df_hwe['n'] * df_hwe['l']

df = pd.concat([df_hwe, df_mcp])

agg = df.groupby(by=['ans', 'n', 'pars']).agg({'grad': {'mean', 'var'}})
agg.columns = agg.columns.to_flat_index()
agg.reset_index(inplace=True)

In [5]:
ax = sns.lineplot(
    data=agg,
    x='pars',
    y=('grad', 'var'),
    hue='n',
    style='ans'
)

legend = ax.get_legend()

leg_dict = {
    'n': 'Qubits',
    'ans': 'Ansatz',
    'hwe': 'HWE',
    'mcp': 'MCP'
}

for text in legend.get_texts():
    s = text.get_text()
    if s in leg_dict:
        text.set_text(leg_dict[s])

ax.set_yscale('log')
ax.set_xlabel('Number of Parameters')
ax.set_ylabel(r'var$(\partial f)$')

Text(0, 0.5, 'var$(\\partial f)$')

<Figure size 432x288 with 1 Axes>

## QAOA

In [81]:
df = pd.read_csv('data/bp_discovery_output_qaoa_20210216_170150.dat')
df['pars'] = 2*df['p']

In [82]:
agg = df.groupby(by=['n', 'pars']).agg({'grad': ['mean', 'var']})
agg.columns = agg.columns.to_flat_index()
agg.reset_index(inplace=True)

In [83]:
ax = sns.lineplot(
    data=agg,
    x='pars',
    y=('grad', 'var'),
    hue='n'
)

ax.legend(loc='lower right')

legend = ax.get_legend()
legend.set_title('Qubits')

ax.set_yscale('log')
ax.set_xlabel('Number of Parameters')
ax.set_ylabel(r'var$(\partial f)$')

Text(0, 0.5, 'var$(\\partial f)$')

<Figure size 432x288 with 1 Axes>

## OCP

In [13]:
df_mcp = pd.read_csv('data/bp_discovery_output_mcp_20210216_150920.dat')
df_mcp['pars'] = df_mcp['l']

df_ocp = pd.read_csv('data/bp_discovery_output_ocp_20210223_121516.dat')
df_ocp['pars'] = df_ocp['l']

In [14]:
df = pd.concat([df_ocp, df_mcp])

agg = df.groupby(by=['ans', 'n', 'pars']).agg({'grad': {'mean', 'var'}})
agg.columns = agg.columns.to_flat_index()
agg.reset_index(inplace=True)

In [15]:
ax = sns.lineplot(
    data=agg,
    x='pars',
    y=('grad', 'var'),
    hue='n',
    style='ans'
)

legend = ax.get_legend()

leg_dict = {
    'n': 'Qubits',
    'ans': 'Ansatz',
    'hwe': 'HWE',
    'mcp': 'MCP',
    'ocp': 'OCP'
}

for text in legend.get_texts():
    s = text.get_text()
    if s in leg_dict:
        text.set_text(leg_dict[s])

ax.set_yscale('log')
ax.set_xlabel('Number of Parameters')
ax.set_ylabel(r'var$(\partial f)$')

Text(0, 0.5, 'var$(\\partial f)$')

<Figure size 432x288 with 1 Axes>

## Hardware Connectivities

In [78]:
df = pd.read_csv('data/bp_discovery_output_hw_conn_20210223_143951.dat')
df['pars'] = df['l']
df['graph_info'] = df['graph_info'].map(ast.literal_eval)
df['name'] = df['graph_info'].map(lambda d: d['name'])

In [79]:
agg = df.groupby(by=['name', 'n', 'pars']).agg({'grad': {'mean', 'var'}})
agg.columns = agg.columns.to_flat_index()
agg.reset_index(inplace=True)
agg.rename(columns={('grad', 'var'): 'var', ('grad', 'mean'): 'mean'}, inplace=True)

In [80]:
g = sns.FacetGrid(agg, col='name', sharex=True)

g.map_dataframe(
    sns.lineplot,
    hue='n',
    x='pars',
    y='var',
)

for name, ax in g.axes_dict.items():
    ax.set_yscale('log')
    ax.set_title(name.title())
    
    ax.set_xlabel('# of Pars')
    ax.set_ylabel(r'var$(\partial f)$')

<Figure size 864x216 with 4 Axes>